# Building a Chatbot using Deep NLP

We have taken dialouges from different movies as our training data, since these are in the form of conversations it'll be helpful in training our network to learn since we building a virtual assistant chatbot.The aim of this chatbot is not just to give answers in "Yes" and "No", but full length answers corresponding to the question asked.

### Importing the libraries

In [ ]:
import numpy as np
import tensorflow as tf
import re # used to clean the text and replace some characters by more simple characters
import time # check the time of each epoch

# Data Preprocessing

### Importing the dataset

In [ ]:
lines = open("movie_lines.txt", encoding='utf-8', errors='ignore' ).read().split('\n')
############## to avoid encoding issue, to ignore any error, to read the dataset, split the observations by the lines
conversations = open("movie_conversations.txt", encoding='utf-8', errors='ignore' ).read().split('\n')

### Creating a dictionary that maps each line and it's ID

Basically what we want is a dataset that contains basically 2 column the input and the output. the I/P will be fed to the Neural Network and the O/P will be the target. The easiest way to do this is with a dictionary, because we can keep track of the conversations.

In [ ]:
id2line = {} # initialize a dictionary
for line in lines:
  _line = line.split(" +++$+++ ")
  if len(_line) == 5:
    id2line[_line[0]] = _line[4]

In [ ]:
id2line

### Creating a list of all the conversations

In [ ]:
conversations_ids = []
for conversation in conversations[:-1]:
   # We wanted to not take the last column since it is empty 
   _conversation = conversation.split(' +++$+++ ')[-1][1:-1].replace("'", "").replace(" ", "")
   conversations_ids.append(_conversation.split(","))

In [ ]:
conversations_ids

### Getting separately the questions and answers

We want two separate lists. One for the questions and one for the answers, but of the same size. This is because, for each index "i" of these lists, the answer of index "i" should be the answer to the question at index "i". Therefore it should be well aligned.

In [ ]:
questions = []
answers = []
for conversation in conversations_ids:
  for i in range(len(conversation)-1):
    questions.append(id2line[conversation[i]])
    answers.append(id2line[conversation[i+1]])

In [ ]:
questions

### Doing a first cleaning of the texts

In [ ]:
def clean_text(text):
  text = text.lower()
  text = re.sub(r"i'm","i am", text)
  text = re.sub(r"he's","he is", text)
  text = re.sub(r"that's","that is", text)
  text = re.sub(r"what's","what is", text)
  text = re.sub(r"where's","where is", text)
  text = re.sub(r"\'ll"," will", text)
  text = re.sub(r"\'ve"," have", text)
  text = re.sub(r"\'re"," are", text)
  text = re.sub(r"\'d"," would", text)
  text = re.sub(r"it's","it is", text)
  text = re.sub(r"won't","will not", text)
  text = re.sub(r"can't","cannot", text)
  text = re.sub(r"[\-\(\)\\\"\#\/\@\;\:\<\>\{\}\+\=\-\|\.\?\,\!]","", text)
  return text

### Cleaning the questions

In [ ]:
clean_questions = []
for question in questions:
  clean_questions.append(clean_text(question))

In [ ]:
clean_questions

### Cleaning the answers

In [ ]:
clean_answers = []
for answer in answers:
  clean_answers.append(clean_text(answer))

In [ ]:
clean_answers

### Creating a dictionary that maps each word to its number of occurences

In [ ]:
word2count = {}
for question in clean_questions:
  for word in question.split():
    if word not in word2count:
      word2count[word] = 1
    else:
      word2count[word] += 1
for answer in clean_answers:
  for word in answer.split():
    if word not in word2count:
      word2count[word] = 1
    else:
      word2count[word] += 1

In [ ]:
word2count

### Tokenization and Filtering the non frequent words:
#### Creating two dictionaries that map the questions words and the answer words to a unique integer

In [ ]:
threshold = 20
questionswords2int = {}
answerswords2int = {}
word_number = 0
for word, count in word2count.items():
  if count >= threshold:
    questionswords2int[word] = word_number
    word_number += 1 
word_number = 0
for word, count in word2count.items():
  if count >= threshold:
    answerswords2int[word] = word_number  
    word_number += 1 

In [ ]:
questionswords2int

In [ ]:
answerswords2int

### Adding the last tokens to these two dictionaries 
#### The start of string i.e. SOS and end of string i.e., EOS. These are useful for the encoder and the decoder in the Seq2Seq model.

In [ ]:
tokens = ['<PAD>', '<EOS>', '<OUT>', '<SOS>']
for token in tokens:
  questionswords2int[token] = len(questionswords2int) +1
for token in tokens:
  answerswords2int[token] = len(answerswords2int) +1

### Creating the inverse dictionary of the answerswords2int dictionary
#### We need to do this because we will need the inverse mapping from the integers to the answers words in the implemnetation of the Seq2Seq model

In [ ]:
answersint2word = {w_i: w for w, w_i in answerswords2int.items()}

In [ ]:
answersint2word

### Adding the End of String token to the end of every answer

In [ ]:
 for i in range(len(clean_answers)):
   clean_answers[i] += " <EOS>"

In [ ]:
clean_answers

### Translating all the questions and the answers into integers and replacing all the words that were filtered out by <OUT>

In [ ]:
questions_into_int = []
for question in clean_questions:
  ints = []
  for word in question.split():
    if word not in questionswords2int:
      ints.append(questionswords2int['<OUT>'])
    else:
      ints.append(questionswords2int[word])
  questions_into_int.append(ints)
answers_into_int = []
for answer in clean_answers:
  ints = []
  for word in answer.split():
    if word not in answerswords2int:
      ints.append(answerswords2int['<OUT>'])
    else:
      ints.append(answerswords2int[word])
  answers_into_int.append(ints)

In [ ]:
questions_into_int

### Sorting questions and answers by the length of questions
#### This will speed up the training and optimize it (reduce padding)

In [ ]:
sorted_clean_questions = []
sorted_clean_answers = []
for length in range(1,30):
  for i in enumerate(questions_into_int):
    if len(i[1]) == length:
      sorted_clean_questions.append(questions_into_int[i[0]])
      sorted_clean_answers.append(answers_into_int[i[0]])    

In [ ]:
sorted_clean_questions

# Building the SEQ2SEQ Model

### Creating placeholders for the inputs and the targets

In [ ]:
def model_inputs():
  inputs = tf.placeholder(tf.int32, [None, None], name = 'input')
  targets = tf.placeholder(tf.int32, [None, None], name= 'target')
  lr = tf.placeholder(tf.float32, name = 'learning_rate')
  keep_prob = tf.placeholder(tf.float32, name = 'keep_prob')
  return inputs, targets, lr, keep_prob

### Preprocessing the Targets
#### We need to do this because the decoder will only accept a certain format of the targets. This format is two-fold. First, the target must be in batches. Secondly, each of the answers in the batch of target must start with the <SOS> token.

In [ ]:
def preprocess_targets(targets, word2int, batch_size):
  left_side = tf.fill([batch_size, 1], word2int["<SOS>"])
  right_side = tf.strided_slice(targets, [0,0], [batch_size, -1], [1,1])
  preprocessed_targets = tf.concat([left_side, right_side], axis = 1)
  return preprocessed_targets

### Creating the Encoder RNN Layer

In [ ]:
# rnn_inputs: model inputs for the rnn
# rnn_size: no. of input tensors of the encoder rnn layer
# keep_prob: we need this to apply dropout regularisation to our lstm
# sequence_length: the list of the length of each question in batch 
def encoder_rnn(rnn_inputs, rnn_size, num_layers, keep_prob, sequence_length):
  # create lstm
  lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
  # apply dropout
  lstm_dropout = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
  encoder_cell = tf.contrib.rnn.MultiRNNCell([lstm_dropout] * num_layers)
  _,encoder_state = tf.nn.bidirectional_dynamic_rnn(cell_fw = encoder_cell, 
                                                    cell_bw = encoder_cell,
                                                    sequence_length = sequence_length,
                                                    inputs = rnn_inputs,
                                                    dtype = tf.float32)
  return encoder_state

### Decoding the training set

In [ ]:
def decode_training_set(encoder_state, decoder_cell, decoder_embedded_input, 
                        sequence_length, decoding_scope, output_function, keep_prob, batch_size):
  attention_states = tf.zeroes([batch_size, 1, decoder_cell.output_size])
  attention_keys, attention_values, attention_score_function, attention_construct_function = tf.contrib.seq2seq.prepare_attention(attention_states, attention_option = "bahdanau", num_units = decoder_cell.output_size)
  # attention_keys: the key to be compared with the target states
  # attention_values: the values we'll use to construct the context vectors
  # attention_score_function: used to compute the similarity between the keys and the target state
  # attention_construct_function: used to build the attention_state
  training_decoder_function = tf.contrib.seq2seq.attention_decoder_fn_train(encoder_state[0], 
                                                                            attention_keys, 
                                                                            attention_values, 
                                                                            attention_score_function, 
                                                                            attention_construct_function,
                                                                            name = 'attn_dec_train')
  decoder_output, decoder_final state, decoder_final_context_state = tf.contrib.seq2seq.dynamic_rnn_decoder(decoder_cell,
                                                                training_decoder_function,
                                                                decoder_embedded_input,
                                                                sequence_length,
                                                                scope = decoding_scope)
  decoder_output_dropout = tf.nn.dropout(decoder_output, keep_prob)
  return output_function(decoder_output_dropout)

### Decoding the test/validation set

In [ ]:
def decode_test_set(encoder_state, decoder_cell, decoder_embeddings_matrix,
                    sos_id, eos_id, maximum_length, num_words, 
                    sequence_length, decoding_scope, output_function, keep_prob, batch_size):
  attention_states = tf.zeroes([batch_size, 1, decoder_cell.output_size])
  attention_keys, attention_values, attention_score_function, attention_construct_function = tf.contrib.seq2seq.prepare_attention(attention_states, attention_option = "bahdanau", num_units = decoder_cell.output_size)
  test_decoder_function = tf.contrib.seq2seq.attention_decoder_fn_inference(output_function, 
                                                                            encoder_state[0], 
                                                                            attention_keys, 
                                                                            attention_values, 
                                                                            attention_score_function, 
                                                                            attention_construct_function,
                                                                            decoder_embeddings_matrix,
                                                                            sos_id, eos_id, 
                                                                            maximum_length, num_words,
                                                                            name = 'attn_dec_inf')
  text_predictions, _, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(decoder_cell,
                                                                  test_decoder_function,
                                                                  scope = decoding_scope)
  return text_predictions

### Creating the Decoder RNN layer

In [ ]:
def decoder_rnn(decoder_embedded_input, decoder_embeddings_matrix,
                encoder_state, num_words, sequence_length, rnn_size, 
                num_layers, word2int, keep_prob, batch_size):
  with tf.variable_scope("decoding") as decoding_scope:
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    lstm_dropout = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
    decoder_cell = tf.contrib.rnn.MultiRNNCell([lstm_dropout] * num_layers)
    weights = tf.truncated_normal_initializer(stddev = 0.1)
    biases = tf.zeroes_initializer()
    output_function = lambda x: tf.contrib.layers.fully_connected(x, num_words, None, 
                                                                  scope = decoding_scope,
                                                                  weights_initializer = weights,
                                                                  biases_initializer = biases)
    training_predictions = decode_training_set(encoder_state, 
                                               decoder_cell,
                                               decoder_embedded_input, sequence_length, 
                                               decoding_scope, output_function, 
                                               keep_prob, batch_size )
    decoding_scope.reuse_variables()
    test_predictions = decode_test_set(encoder_state, 
                                       decoder_cell, 
                                       decoder_embeddings_matrix, 
                                       word2int['<SOS>'], word2int['<EOS>'] , 
                                       sequence_length-1, 
                                       num_words, 
                                       sequence_length, decoding_scope, 
                                       output_function, keep_prob, batch_size)
    return training_predictions, test_predictions

### Building the Seq2Seq model

In [ ]:
def seq2seq_model(inputs, targets, keep_prob, batch_size, sequence_length, 
                  answers_num_words, questions_num_words,
                  encoder_embedding_size, decoder_embedding_size,
                  rnn_size, num_layers, questionswords2int):
  encoder_embedded_input = tf.contrib.layers.embed_sequence(inputs, 
                                                            answers_num_words +1,
                                                            encoder_embedding_size,
                                                            initializer = tf.random_uniform_initializer(0,1))
  encoder_state = encoder_rnn(encoder_embedded_input, rnn_size, num_layers, keep_prob, sequence_length)
  preprocessed_targets = preprocess_targets(targets, questionsword2int, batch_size)
  decoder_embeddings_matrix = tf.Variable(tf.random_normal_initializer([questions_num_words +1, decoder_embedding_size], 0, 1))
  decoder_embedded_input = tf.nn.embedding_lookup(decoder_embeddings_matrix, preprocess_targets )
  training_predictions, test_predictions = decoder_rnn(decoder_embedded_input, 
                                                       decoder_embeddings_matrix,
                                                       encoder_state, 
                                                       questions_num_words, 
                                                       sequence_length, rnn_size, 
                                                       num_layers, 
                                                       questionswords2int, 
                                                       keep_prob, batch_size)
  return training_predictions, test_predictions

# Training the SEQ2SEQ Model

### Setting the Hyperparameters

In [ ]:
epochs = 100
batch_size = 32
rnn_size = 1024
num_layers = 3
encoding_embedding_size = 1024
decoding_embedding_size = 1024
learning_rate = 0.001
learning_rate_decay = 0.9
min_learning_rate = 0.0001
keep_probability = 0.5

### Defining a session

In [ ]:
tf.reset_default_graph()
session = tf.InteractiveSession()

### Loading the model inputs

In [ ]:
inputs, targets, lr, keep_prob = model_inputs()

### Setting the sequence length

In [ ]:
sequence_length = tf.placeholder_with_default(25, None, name = 'sequence_length')

### Getting the shape of the inputs tensor

In [ ]:
input_shape = tf.shape(inputs)

### Getting the shape of the inputs tensor

In [ ]:
input_shape = tf.shape(inputs)

### Getting the training and test predictions

In [ ]:
training_predictions, test_predictions = seq2seq_model(tf.reverse(inputs, [-1]),
                                                       targets,
                                                       keep_prob,
                                                       batch_size,
                                                       sequence_length,
                                                       len(answerswords2int),
                                                       len(questionswords2int),
                                                       encoding_embedding_size,
                                                       decoding_embedding_size,
                                                       rnn_size,
                                                       num_layers,
                                                       questionswords2int)

### Setting up the Loss Error, the Optimizer and Gradient Clipping

In [ ]:
with tf.name_scope("optimization"):
    loss_error = tf.contrib.seq2seq.sequence_loss(training_predictions,
                                                  targets,
                                                  tf.ones([input_shape[0], sequence_length]))
    optimizer = tf.train.AdamOptimizer(learning_rate)
    gradients = optimizer.compute_gradients(loss_error)
    clipped_gradients = [(tf.clip_by_value(grad_tensor, -5., 5.), grad_variable) for grad_tensor, grad_variable in gradients if grad_tensor is not None]
    optimizer_gradient_clipping = optimizer.apply_gradients(clipped_gradients)

### Padding the sequences with the <PAD> token

In [ ]:
def apply_padding(batch_of_sequences, word2int):
    max_sequence_length = max([len(sequence) for sequence in batch_of_sequences])
    return [sequence + [word2int['<PAD>']] * (max_sequence_length - len(sequence)) for sequence in batch_of_sequences]

### Splitting the data into batches of questions and answers

In [ ]:
def split_into_batches(questions, answers, batch_size):
    for batch_index in range(0, len(questions) // batch_size):
        start_index = batch_index * batch_size
        questions_in_batch = questions[start_index : start_index + batch_size]
        answers_in_batch = answers[start_index : start_index + batch_size]
        padded_questions_in_batch = np.array(apply_padding(questions_in_batch, questionswords2int))
        padded_answers_in_batch = np.array(apply_padding(answers_in_batch, answerswords2int))
        yield padded_questions_in_batch, padded_answers_in_batch

### Splitting the questions and answers into training and validation sets

In [ ]:
training_validation_split = int(len(sorted_clean_questions) * 0.15)
training_questions = sorted_clean_questions[training_validation_split:]
training_answers = sorted_clean_answers[training_validation_split:]
validation_questions = sorted_clean_questions[:training_validation_split]
validation_answers = sorted_clean_answers[:training_validation_split]

### Training

In [ ]:
batch_index_check_training_loss = 100
batch_index_check_validation_loss = ((len(training_questions)) // batch_size // 2) - 1
total_training_loss_error = 0
list_validation_loss_error = []
early_stopping_check = 0
early_stopping_stop = 100
checkpoint = "chatbot_weights.ckpt"
session.run(tf.global_variables_initializer())
for epoch in range(1, epochs + 1):
    for batch_index, (padded_questions_in_batch, padded_answers_in_batch) in enumerate(split_into_batches(training_questions, training_answers, batch_size)):
        starting_time = time.time()
        _, batch_training_loss_error = session.run([optimizer_gradient_clipping, loss_error], {inputs: padded_questions_in_batch,
                                                                                               targets: padded_answers_in_batch,
                                                                                               lr: learning_rate,
                                                                                               sequence_length: padded_answers_in_batch.shape[1],
                                                                                               keep_prob: keep_probability})
        total_training_loss_error += batch_training_loss_error
        ending_time = time.time()
        batch_time = ending_time - starting_time
        if batch_index % batch_index_check_training_loss == 0:
            print('Epoch: {:>3}/{}, Batch: {:>4}/{}, Training Loss Error: {:>6.3f}, Training Time on 100 Batches: {:d} seconds'.format(epoch,
                                                                                                                                       epochs,
                                                                                                                                       batch_index,
                                                                                                                                       len(training_questions) // batch_size,
                                                                                                                                       total_training_loss_error / batch_index_check_training_loss,
                                                                                                                                       int(batch_time * batch_index_check_training_loss)))
            total_training_loss_error = 0
        if batch_index % batch_index_check_validation_loss == 0 and batch_index > 0:
            total_validation_loss_error = 0
            starting_time = time.time()
            for batch_index_validation, (padded_questions_in_batch, padded_answers_in_batch) in enumerate(split_into_batches(validation_questions, validation_answers, batch_size)):
                batch_validation_loss_error = session.run(loss_error, {inputs: padded_questions_in_batch,
                                                                       targets: padded_answers_in_batch,
                                                                       lr: learning_rate,
                                                                       sequence_length: padded_answers_in_batch.shape[1],
                                                                       keep_prob: 1})
                total_validation_loss_error += batch_validation_loss_error
            ending_time = time.time()
            batch_time = ending_time - starting_time
            average_validation_loss_error = total_validation_loss_error / (len(validation_questions) / batch_size)
            print('Validation Loss Error: {:>6.3f}, Batch Validation Time: {:d} seconds'.format(average_validation_loss_error, int(batch_time)))
            learning_rate *= learning_rate_decay
            if learning_rate < min_learning_rate:
                learning_rate = min_learning_rate
            list_validation_loss_error.append(average_validation_loss_error)
            if average_validation_loss_error <= min(list_validation_loss_error):
                print('I speak better now!!')
                early_stopping_check = 0
                saver = tf.train.Saver()
                saver.save(session, checkpoint)
            else:
                print("Sorry I do not speak better, I need to practice more.")
                early_stopping_check += 1
                if early_stopping_check == early_stopping_stop:
                    break
    if early_stopping_check == early_stopping_stop:
        print("My apologies, I cannot speak better anymore. This is the best I can do.")
        break
print("Game Over")

# Testing the SEQ2SEQ Model

### Loading the weights and Running the session

In [ ]:
checkpoint = "./chatbot_weights.ckpt"
session = tf.InteractiveSession()
session.run(tf.global_variables_initializer())
saver = tf.train.Saver()
saver.restore(session, checkpoint)

### Converting the questions from strings to lists of encoding integers

In [ ]:
def convert_string2int(question, word2int):
    question = clean_text(question)
    return [word2int.get(word, word2int['<OUT>']) for word in question.split()]

### Setting up the chat

In [ ]:
while(True):
    question = input("You: ")
    if question == 'Goodbye':
        break
    question = convert_string2int(question, questionswords2int)
    question = question + [questionswords2int['<PAD>']] * (25 - len(question))
    fake_batch = np.zeros((batch_size, 25))
    fake_batch[0] = question
    predicted_answer = session.run(test_predictions, {inputs: fake_batch, keep_prob: 0.5})[0]
    answer = ''
    for i in np.argmax(predicted_answer, 1):
        if answersints2word[i] == 'i':
            token = ' I'
        elif answersints2word[i] == '<EOS>':
            token = '.'
        elif answersints2word[i] == '<OUT>':
            token = 'out'
        else:
            token = ' ' + answersints2word[i]
        answer += token
        if token == '.':
            break
    print('ChatBot: ' + answer)